In [ ]:
# Load the Drive helper and mount
from google.colab import drive
import os

# This will prompt for authorization.
drive.mount('/content/drive') 

Mounted at /content/drive


# New Section

In [ ]:
os.chdir('/content/drive/My Drive/Humana')
# os.listdir('Data')

In [ ]:
!pip install pytorch_tabnet
!pip install catboost

     |████████████████████████████████| 76.4 MB 28 kB/s 


In [ ]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

# Data Cleaning

In [1]:
import pandas as pd 
import numpy as np 
df_train=pd.read_csv('Data/2021_Competition_Training.csv')
df_train=reduce_mem_usage(df_train)


df_test=pd.read_csv('Data/2021_Competition_Holdout.csv')
df_test=reduce_mem_usage(df_test)

df=pd.concat((df_train,df_test))

del df_train, df_test

df['covid_vaccination']=(df['covid_vaccination']=='vacc').astype(int)

target='covid_vaccination'

display(df.head())
df.info()

## Seperate numeric feature and categorical feature

In [ ]:
# cat fea but type is int
cat_feas=['bh_ncal_ind', 'bh_ncdm_ind']    

# num fea but type is str
num_feas=['rx_gpi2_34_dist_gpi6_pmpm_ct','rx_gpi2_33_pmpm_ct_0to3m_b4','src_div_id',
              'lab_albumin_loinc_pmpm_ct','ccsp_065_pmpm_ct','rev_cms_ansth_pmpm_ct',
              'rej_total_physician_office_visit_ct_pmpm_0to3m_b4','rx_gpi2_90_dist_gpi6_pmpm_ct_9to12m_b4',
              'rx_gpi2_56_dist_gpi6_pmpm_ct_3to6m_b4','rx_gpi2_72_pmpm_ct_6to9m_b4','rx_bh_pmpm_ct_0to3m_b4','rx_gpi4_6110_pmpm_ct']

for col in df.columns:
    if col !=target:
        dtype=df[col].dtype
        if col in num_feas:
            df[col]=df[col].replace('*',None).astype(float)
        if col not in num_feas+cat_feas+[target]:
            if dtype=='O':
                cat_feas.append(col)
            else:
                num_feas.append(col)

cat_feas=[x for x in cat_feas if 'zip_cd' not in x]
num_feas=[x for x in num_feas if 'zip_cd' not in x]


for col in num_feas+cat_feas:
    if col in cat_feas:
        if df[col].isnull().sum():
            df[col]=df[col].fillna('null')
        df[col]=df[col].astype(str).astype('category')#.cat.codes
    else:
        m=df[col].astype(float).mean()
        df[col]=df[col].fillna(m)

In [ ]:
df=reduce_mem_usage(df)

df[df[target]!='null'][cat_feas+num_feas+[target]].to_csv('Data/df_train_1007.csv',index=False)
df[df[target]=='null'][cat_feas+num_feas].to_csv('Data/df_val_1007.csv',index=False)


Mem. usage decreased to 936.96 Mb (10.8% reduction)


In [ ]:
import pandas as pd 
import numpy as np

target='covid_vaccination'

df_train=pd.read_csv('Data/df_train_1007.csv')
df_train=reduce_mem_usage(df_train)

df_val=df_train.iloc[974842:,:-1]
df_train=df_train.iloc[:974842,:]

print(df_train.info())
print(df_val.info())

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (3,4,5,7,9,12,13,14,15,19,21,22,24,25,26,27,31,32,39,40,41,42,46,49,50,52,55,59,62,63,64,66,68,70,74,75,76,77,83,86,87,89,90,91,92,93,99,101,102,103,104,105,106) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Mem. usage decreased to 1925.47 Mb (54.2% reduction)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 974842 entries, 0 to 974841
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: float16(206), float32(8), int16(4), int32(1), int8(42), object(106)
memory usage: 1.2+ GB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525158 entries, 974842 to 1499999
Columns: 366 entries, bh_ncal_ind to race_cd
dtypes: float16(206), float32(8), int16(4), int32(1), int8(41), object(106)
memory usage: 673.6+ MB
None


In [ ]:
for col in cat_feas:
    df_train[col]=df_train[col].astype(str).astype('category')
    df_val[col]=df_val[col].astype(str).astype('category')


# Run Model

Our dataset is an imbalanced dataset, 17% of the total sample are positive samples. In other words, the ratio of the number of positive samples to negative samples is 1:5. 

To solve this problem, we used methods that are similar to down-sampling. We randomly split all the negative samples into 5 parts and for each part, we combine it with all the positive samples. Therefore, we get 5 new data sets, each of them containing 161,077 negative samples and 169,453 positive samples, approximately a 1:1 ratio and can be treated as a balanced dataset.


During the training stage, we train our models on five new datasets separately with 10 folds cross validation. This will give us 50 models in total. For the prediction, we use the blending methods to ensemble all 50 models. This will not only utilize all the sample information, but also significantly improve the model's robustness.

In [ ]:
pos_ind=df_train[df_train[target]==1].index.tolist()
neg_ind=df_train[df_train[target]==0].index.tolist()

np.random.shuffle(neg_ind)

neg_inds=[]

for i in range(5):
    neg_inds.append(neg_ind[len(pos_ind)*i:len(pos_ind)*(i+1)])


In [ ]:
# %%time
from catboost import CatBoostClassifier, Pool,EShapCalcType, EFeaturesSelectionAlgorithm
import pickle
from pytorch_tabnet.metrics import Metric
from pytorch_tabnet.tab_model import TabNetClassifier
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
import torch 

from sklearn.metrics import average_precision_score,f1_score,accuracy_score,roc_curve,auc
from IPython.display import clear_output


folds=5

res=[]
for i,neg_ind in enumerate(neg_inds):

    print('\n\n','#'*30,'subset',i,'#'*30,'\n')
    df_fast_check=df_train.loc[pos_ind+neg_ind]
    
    # shuffle data
    df_fast_check['ran']=[np.random.random() for _ in range(len(df_fast_check))]
    df_fast_check=df_fast_check.sort_values('ran').reset_index()
    df_fast_check.drop(['ran','index'],axis=1,inplace=True)

    for fold in range(folds):
        sample_in_fold=len(df_fast_check)//folds
        df_fast_check_test=df_fast_check.iloc[sample_in_fold*fold:sample_in_fold*(fold+1)]
        
        print(df_fast_check_test.info())

        df_fast_check_train=df_fast_check.loc[set(df_fast_check.index)-set(df_fast_check_test.index)]
        print(df_fast_check_train.info())

        train_data=df_fast_check_train[cat_feas+num_feas]
        train_labels=df_fast_check_train[target]
        test_data=df_fast_check_test[cat_feas+num_feas]
        test_labels=df_fast_check_test[target]
        
        # test_labels.to_csv('Data/subset_test_label_v2_%s.csv'%i,index=False)


        ####################################### TabNet ########################################
        tabnet_params = dict(n_d=24, n_a=24, gamma=1.3,
                                optimizer_fn=torch.optim.Adam, # tabnet package is based on torch, so tf.keras.optimizers.Adam will lead to an error
                                optimizer_params=dict(lr=1e-2),
                                scheduler_params=dict(mode="min",
                                                    patience=5,
                                                    min_lr=1e-5,
                                                    factor=0.9,),
                                scheduler_fn=torch.optim.lr_scheduler.ReduceLROnPlateau,   # same, here we can't use tf.keras.callbacks.ReduceLROnPlateau,
                                verbose=1,
                                # cat_idxs=range(len(num_feas),len(cat_feas+num_feas))
                                )

        # tabnet_params = dict()        

        X_train=train_data.values
        y_train=train_labels.values
        
        X_test=test_data.values
        y_test=test_labels.values

        model = TabNetClassifier(**tabnet_params)
        model.fit(X_train=X_train,
                    y_train=y_train,
                    eval_set=[(X_test, y_test)],
                    eval_name=["val"],
                    eval_metric=['accuracy','auc'],
                    # max_epochs=1,
                    max_epochs=250,
                    patience=15, 
                    # batch_size=512, 
                    # virtual_batch_size=64,
                    # loss_fn=torch.nn.functional.binary_cross_entropy,
                    ) 

        torch.save(model, 'Model/tabnet_%s.pt'%i)
        # pickle.dump(model, open('Model/tabnet_red_%s'%i, 'wb'))
        
        preds_class = model.predict_proba(X_test)
        np.save('Data/tab_test_%s.npy'%i,preds_class[:,1])

        preds_class = model.predict_proba(X_val)
        np.save('Data/tab_val_%s.npy'%i,preds_class[:,1])



        ######################################## CatBoost ########################################
        print('\n\n')
        catboost_pool = Pool(test_data, 
                                        test_labels,
                                        # cat_features=imp_cat,
                                        cat_features=cat_feas,
                                            )

        model_cat = CatBoostClassifier(
                                    # iterations=1,
                                    iterations=1500,
                                    learning_rate=0.1,
                                    cat_features=cat_feas,
                                    # cat_features=imp_cat,
                                    task_type="GPU",
                                    early_stopping_rounds=100,
                                    depth=3,
                                    # eval_metric=['AUC','F1','Accuracy'],
                                    #    loss_function='Logloss',
                                    # verbose=False,
                                    verbose=50,
                                    )

        # train the model_cat
        model_cat.fit(train_data, train_labels,eval_set=catboost_pool,use_best_model=True)

        # pickle.dump(model_cat, open('Model/catboost_red_%s'%i, 'wb'))
        model.save_model('Model/catboost_%s'%i, format="python")

        preds_class = model_cat.predict_proba(test_data)
        np.save('Data/1007_cat_test_%s.npy'%i,preds_class[:,1])

        preds_class = model_cat.predict_proba(df_val[cat_feas+num_feas])
        np.save('Data/1007_cat_val_%s.npy'%i,preds_class[:,1])

        ev=model_cat.eval_metrics(catboost_pool,['AUC','F1','Accuracy'])

        res=[]
        for key in ev.keys():
            res.append([key,max(ev[key]),ev[key][-1]])

        display(pd.DataFrame(res,columns=['metirc','max','last']))

        # break

    # break



 ############################## subset 0 ############################## 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67781 entries, 0 to 67780
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 86.7 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 271125 entries, 67781 to 338905
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 206.5 MB
None



0:	learn: 0.6892731	test: 0.6894797	best: 0.6894797 (0)	total: 204ms	remaining: 5m 5s
50:	learn: 0.6493276	test: 0.6520859	best: 0.6520859 (50)	total: 8.15s	remaining: 3m 51s
100:	learn: 0.6428002	test: 0.6461771	best: 0.6461771 (100)	total: 16s	remaining: 3m 42s
150:	learn: 0.6401527	test: 0.6440241	best: 0.6440241 (150)	total: 23.8s	remaining: 3m 33s
200:	learn: 0.6385345	test: 0.6428517	best: 0.6428517 (200)	total: 31.6s	remaining: 3m 

,metirc,max,last
0,AUC,0.678727,0.678710
1,F1,0.658014,0.648701
2,Accuracy,0.629896,0.629586




 ############################## subset 1 ############################## 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67781 entries, 0 to 67780
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 86.7 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 271125 entries, 67781 to 338905
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 206.5 MB
None



0:	learn: 0.6892699	test: 0.6893392	best: 0.6893392 (0)	total: 199ms	remaining: 4m 58s
50:	learn: 0.6494127	test: 0.6496435	best: 0.6496435 (50)	total: 8.14s	remaining: 3m 51s
100:	learn: 0.6438579	test: 0.6442498	best: 0.6442498 (100)	total: 16.1s	remaining: 3m 42s
150:	learn: 0.6410890	test: 0.6417086	best: 0.6417086 (150)	total: 23.8s	remaining: 3m 32s
200:	learn: 0.6394755	test: 0.6403918	best: 0.6403918 (200)	total: 31.6s	remaining: 

,metirc,max,last
0,AUC,0.684074,0.684074
1,F1,0.657128,0.651662
2,Accuracy,0.635119,0.634883




 ############################## subset 2 ############################## 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67781 entries, 0 to 67780
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 86.7 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 271125 entries, 67781 to 338905
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 206.5 MB
None



0:	learn: 0.6892234	test: 0.6893619	best: 0.6893619 (0)	total: 200ms	remaining: 4m 59s
50:	learn: 0.6498821	test: 0.6502383	best: 0.6502383 (50)	total: 8.13s	remaining: 3m 50s
100:	learn: 0.6441332	test: 0.6449938	best: 0.6449938 (100)	total: 15.9s	remaining: 3m 40s
150:	learn: 0.6410539	test: 0.6423443	best: 0.6423443 (150)	total: 23.6s	remaining: 3m 31s
200:	learn: 0.6394983	test: 0.6411932	best: 0.6411932 (200)	total: 31.3s	remaining: 

,metirc,max,last
0,AUC,0.681398,0.681390
1,F1,0.652309,0.651868
2,Accuracy,0.632626,0.632345




 ############################## subset 3 ############################## 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67781 entries, 0 to 67780
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 86.7 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 271125 entries, 67781 to 338905
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 206.5 MB
None



0:	learn: 0.6894240	test: 0.6894203	best: 0.6894203 (0)	total: 204ms	remaining: 5m 6s
50:	learn: 0.6496413	test: 0.6496454	best: 0.6496454 (50)	total: 8.13s	remaining: 3m 50s
100:	learn: 0.6439517	test: 0.6442260	best: 0.6442260 (100)	total: 15.9s	remaining: 3m 40s
150:	learn: 0.6412893	test: 0.6419200	best: 0.6419200 (150)	total: 23.6s	remaining: 3m 31s
200:	learn: 0.6397012	test: 0.6407156	best: 0.6407156 (200)	total: 31.3s	remaining: 3

,metirc,max,last
0,AUC,0.682784,0.682783
1,F1,0.652034,0.651506
2,Accuracy,0.632124,0.631696




 ############################## subset 4 ############################## 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59406 entries, 0 to 59405
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 81.9 MB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 237624 entries, 59406 to 297029
Columns: 367 entries, bh_ncal_ind to covid_vaccination
dtypes: category(108), float16(206), float32(8), int16(4), int32(1), int8(40)
memory usage: 186.8 MB
None



0:	learn: 0.6873202	test: 0.6874847	best: 0.6874847 (0)	total: 167ms	remaining: 4m 10s
50:	learn: 0.6392251	test: 0.6404686	best: 0.6404686 (50)	total: 6.89s	remaining: 3m 15s
100:	learn: 0.6334827	test: 0.6348062	best: 0.6348062 (100)	total: 13.5s	remaining: 3m 7s
150:	learn: 0.6308063	test: 0.6323788	best: 0.6323788 (150)	total: 20s	remaining: 2m 59s
200:	learn: 0.6291950	test: 0.6310375	best: 0.6310375 (200)	total: 26.6s	remaining: 2m 

,metirc,max,last
0,AUC,0.683433,0.683433
1,F1,0.730815,0.727275
2,Accuracy,0.652409,0.651517
